In [1]:
import scipy.io as scio
import matplotlib.pyplot as plt
import numpy as np
import random
import warnings
import datetime
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import confusion_matrix
from skbayes.rvm_ard_models import RVC

In [2]:
warnings.filterwarnings("ignore")
train_data = scio.loadmat('Proj2FeatVecsSet1.mat')['Proj2FeatVecsSet1']
output_labels = scio.loadmat('Proj2TargetOutputsSet1.mat')['Proj2TargetOutputsSet1']
print(train_data.shape, output_labels.shape)

(25000, 60) (25000, 5)


In [3]:
def encode_output_labels(output_labels=output_labels):
    "Encodes output labels to a single value"
    out_labels = []
    for row in output_labels:
        out_labels.append(np.where(row == 1)[0][0])
    return np.array(out_labels)

def generate_unknown_class_data(entries=10000):
    "Returns Data and Output Labels for Unknown Class(Nc+1)"
    unknown_class_data = []
    for i in range(entries):
        rand = []
        for j in range(train_data.shape[1]):
            rand.append(random.random())
        unknown_class_data.append(rand)
    unknown_class_label = np.full((10000), 5, dtype=int)
    return np.array(unknown_class_data), unknown_class_label

#Funtion K_Fold_Seperation(Xtrain,Ytrain,Nf)
#    Use K_Fold to seperate the training set into Nf parts, take Nf-1 parts as Estimation set
#    Take the remaining 1 as Validation set
#    Inputs:
#           Xtrain: Training Dataset Feature Vectors
#           Ytrain: Training Dataset Label Vectors
#           Nf:     Number of Folds for K_Fold Cross Validation
#    Outputs:
#           x_est_def: Estimation Sets Feature Vectors, Size = 5
#           x_val_def: Validation Sets Feature Vectors, Size = 5
#           y_est_def: Estimation Sets Label Vectors, Size = 5
#           y_val_def: Validation Sets Label Vectors, Size = 5
def K_Fold_Seperation(Xtrain,Ytrain,Nf):
    "Separate the Training Data into Nf Folds and Generate Estimation and Validation Sets"
    x_est_def = []
    x_val_def = []
    y_est_def = []
    y_val_def = []

    kf = KFold(n_splits=Nf)
    for train,test in kf.split(Xtrain):
        x_est_def.append(Xtrain[train])
        x_val_def.append(Xtrain[test])
        y_est_def.append(Ytrain[train])
        y_val_def.append(Ytrain[test])

    x_est_def = np.array(x_est_def)
    x_val_def = np.array(x_val_def)
    y_est_def = np.array(y_est_def)
    y_val_def = np.array(y_val_def)
    return x_est_def, x_val_def, y_est_def, y_val_def


In [4]:
# Encode Output Lables to a single value
out_labels = encode_output_labels(output_labels=output_labels)

# Generate Data for Training Unknown Class Nc+1
unknown_class_data, unknown_class_labels = generate_unknown_class_data(entries=10000)

# Merge Actual and Generated Class Nc+1 Training Data and
train_data = np.concatenate((train_data, unknown_class_data))
out_labels = np.concatenate((out_labels, unknown_class_labels))

print(train_data.shape, out_labels.shape)

(35000, 60) (35000,)


In [5]:
x_train_def, x_test_def, y_train_def, y_test_def = train_test_split(train_data, out_labels,
                                                                   test_size=0.33,
                                                                   shuffle=True)
x_train_def = x_train_def[:5000]
x_test_def = x_test_def[:5000]
y_train_def = y_train_def[:5000]
y_test_def = y_test_def[:5000]

# Apply K_Fold to Training Dataset Feature & Label Vectors
# Get Estimation Sets and Validation Sets
x_est_def, x_val_def, y_est_def, y_val_def = K_Fold_Seperation(x_train_def, y_train_def, 5)
print(x_est_def.shape, x_val_def.shape, y_est_def.shape, y_val_def.shape)
print(x_test_def.shape, y_test_def.shape)

(5, 4000, 60) (5, 1000, 60) (5, 4000) (5, 1000)
(5000, 60) (5000,)


In [6]:
#Funtion svm_cv,rvm_cv, and gp_cv(x_train, y_train, algorithm_parameters, Nf)
#    Use GridSearchCV and Cross Validation to Find the Best Fitted Model of Each Algorithm
#    The Returned Model Contains EstParameters
#    Inputs:
#           Xtrain: Training Dataset Feature Vectors
#           Ytrain: Training Dataset Label Vectors
#           algorithm_parameters: Candidate Parameters for Each Algorithm
#           Nf:     Number of Folds for K_Fold Cross Validation
#    Outputs:
#           cvsvm: Best Fitted Model of SVM
#           cvrvm: Best Fitted Model of RVM
#           cvgpc: Best Fitted Model of GPR

# Define parameter candidates to get the optimal SVM model
# svm_parameters = {'kernel':('linear', 'poly', 'rbf'), 'gamma':[0.125, 0.5, 1, 4], 'decision_function_shape':['ovo','ovr'],
#                  'tol':[1e-3, 5e-3, 1e-2]}
svm_parameters = {'gamma':[0.25, 1, 4], 'tol':[0.001,0.005]}
def svm_cv(x_train, y_train,
                 svm_parameters, Nf):
    """Trains a Support Vector Machine Classifer"""
    # Create Model with passed hyperparameters
    svc = SVC()
    cvsvm = GridSearchCV(svc, svm_parameters, cv=Nf)
    
    # Train Model 
    cvsvm.fit(x_train,y_train)
    
    # Return Trained Model and Accuracy on Test Data
    return cvsvm

# Define parameter candidates to get the optimal RVM model
# rvm_parameters = {'kernel':('linear', 'poly', 'rbf'), 'degree':[2, 3], 'n_iter':[100, 200], 'tol':[0.001,0.005]}
rvm_parameters = {'kernel':('linear', 'poly', 'rbf'), 'tol':[0.001,0.005]}
def rvm_cv(x_train,y_train,
                 rvm_parameters, Nf):
    """Trains a Relevance Vector Machine Classifier"""
    
    rvm = RVC()
    cvrvm = GridSearchCV(rvm, rvm_parameters, cv=Nf)
    
    # Train Model 
    cvrvm.fit(x_train,y_train)    
    
    # Return Trained Model and Accuracy on Test Data
    return cvrvm

# Define parameter candidates to get the optimal GPR model
# gp_parameters = {'n_restarts_optimizer':[0,1], 'max_iter_predict':[50, 100], 'warm_start':('True','False'),
#                'multi_class':('one_vs_rest','one_vs_one')}
gp_parameters = {'n_restarts_optimizer':[0,2], 'max_iter_predict':[50, 100]}
def gp_cv(x_train,y_train,
                gp_parameters, Nf):
    """Trains a Gaussian Process Classifier"""
    
    gpc = GaussianProcessClassifier()
    cvgpc = GridSearchCV(gpc, gp_parameters, cv=Nf)
    
    # Train Model 
    cvgpc.fit(x_train, y_train)

    # Return Trained Model and Accuracy on Test Data
    return cvgpc

In [7]:
#Funtion MyCrossValidate(XTrain, YTrain, Nf)
#    Use cross validation to get the optimal parameters and hyper-parameters
#    Use the trained model to get confusion-matrix of each validation set
#    And the confusion-matrix for the whole training set
#    Return Ytrain, EstParameters, EstConfMatrices, and ConfMatrix
#    Inputs:
#           Xtrain: Training Dataset Feature Vectors
#           Ytrain: Training Dataset Label Vectors
#           Nf:     Number of Folds for K_Fold Cross Validation
#    Outputs:
#           SVMY_Pred,RVMY_Pred,GPY_Pred: Class labels for each fold, corresponds to Ytrain
#           cvsvm.best_params_,cvrvm.best_params_,cvgpc.best_params_: Best Parameters, corresponds to EstParameters
#           SVMEstConfMat,RVMEstConfMat,GPEstConfMat: Confusion Matrices for each fold, corresponds to EstConfMatrices
#           SVMConfMatrix,RVMConfMatrix,GPConfMatrix: Overall Confusion Matrix, corresponds to ConfMatrix
def MyCrossValidate(XTrain, YTrain, Nf):
    cvsvm = svm_cv(XTrain, YTrain, svm_parameters, Nf)
    cvrvm = rvm_cv(XTrain, YTrain, rvm_parameters, Nf)
    cvgpc = gp_cv(XTrain, YTrain, gp_parameters, Nf)
    
    SVMEstConfMat = []
    SVMY_Pred = []
    RVMEstConfMat = []
    RVMY_Pred = []
    GPEstConfMat = []
    GPY_Pred = []
    
    # Best Models for SVM and RVM acquired through Cross Validation
    bestsvm = cvsvm.best_estimator_
    bestrvm = cvrvm.best_estimator_
    
    for i in range(x_val_def.shape[0]):
        # For each validation sets, predict the labels and get its confusion matrix  
        svmtemp_pred = cvsvm.predict(x_val_def[i])
        svmtemp_conf = confusion_matrix(y_val_def[i], svmtemp_pred)
        rvmtemp_pred = cvrvm.predict(x_val_def[i])
        rvmtemp_conf = confusion_matrix(y_val_def[i], rvmtemp_pred)
        gptemp_pred = cvgpc.predict(x_val_def[i])
        gptemp_conf = confusion_matrix(y_val_def[i], gptemp_pred)
        
        bestsvm.fit(x_est_def[i],y_est_def[i])
        bestsvm.predict(x_val_def[i])
        print('The Shape of Support Vectors Array of SVM model for Fold %d is:' % i )
        print(bestsvm.support_vectors_.shape)
        
        bestrvm.fit(x_est_def[i],y_est_def[i])
        bestrvm.predict(x_val_def[i])
        print('The Length of Relevance Vectors Array of RVM model for Fold %d is:' % i )
        print(len(bestrvm.relevant_vectors_))

        SVMEstConfMat.append(svmtemp_conf)
        SVMY_Pred.append(svmtemp_pred)
        RVMEstConfMat.append(rvmtemp_conf)
        RVMY_Pred.append(rvmtemp_pred)
        GPEstConfMat.append(gptemp_conf)
        GPY_Pred.append(gptemp_pred)
        
    SVMEstConfMat = np.array(SVMEstConfMat)
    SVMY_Pred = np.array(SVMY_Pred)
    RVMEstConfMat = np.array(RVMEstConfMat)
    RVMY_Pred = np.array(RVMY_Pred)
    GPEstConfMat = np.array(GPEstConfMat)
    GPY_Pred = np.array(GPY_Pred)
    
    YTrue = y_val_def.flatten()
    SVM_All_pred = SVMY_Pred.flatten()    
    SVMConfMatrix = confusion_matrix(YTrue, SVM_All_pred)
    RVM_All_pred = RVMY_Pred.flatten()
    RVMConfMatrix = confusion_matrix(YTrue, RVM_All_pred)
    GP_All_pred = GPY_Pred.flatten()
    GPConfMatrix = confusion_matrix(YTrue, GP_All_pred)
    
    return SVMY_Pred,RVMY_Pred,GPY_Pred,cvsvm.best_params_,cvrvm.best_params_,cvgpc.best_params_,\
SVMEstConfMat,RVMEstConfMat,GPEstConfMat,SVMConfMatrix,RVMConfMatrix,GPConfMatrix


In [8]:
#Funtion Dis_CV_Info(SVM_best_params,RVM_best_params,GPR_best_params,
#SVMEstConfMat,RVMEstConfMat,GPEstConfMat,SVMConfMatrix,RVMConfMatrix,GPConfMatrix)
#    Display the info acquired through Cross Validation
#    Display EstParameters, EstConfMatrices, and ConfMatrix
#    Inputs:
#           SVM_best_params,RVM_best_params,GPR_best_params: EstParameters of Each Algorithm
#           SVMEstConfMat,RVMEstConfMat,GPEstConfMat: EstConfMatrices of Each Algorithm
#           SVMConfMatrix,RVMConfMatrix,GPConfMatrix: ConfMatrix of Each Algorithm
#    Outputs:
#           None
def Dis_CV_Info(SVM_best_params,RVM_best_params,GPR_best_params,
SVMEstConfMat,RVMEstConfMat,GPEstConfMat,SVMConfMatrix,RVMConfMatrix,GPConfMatrix):
    print('The parameters of the best SVM model are: ')
    print(SVM_best_params)
    print('The parameters of the best RVM model are: ')
    print(RVM_best_params)
    print('The parameters of the best GPR model are: ')
    print(GPR_best_params)
    
    for i in range(x_val_def.shape[0]):
        print('The Confusion Matrix For Fold %d is: ' % i)
        print('SVM:')
        print(SVMEstConfMat[i])
        print('RVM:')
        print(RVMEstConfMat[i])
        print('GPR:')
        print(GPEstConfMat[i])

    print('The Overall Confusion Matrix is: ')
    print('SVM:')
    print(SVMConfMatrix)
    print('RVM:')
    print(RVMConfMatrix)
    print('GPR:')
    print(GPConfMatrix)
    return 1

In [9]:
# SVM_Y,RVM_Y,GPR_Y corresponds to Ytrain
# SVM_best_params,RVM_best_params,GPR_best_params corresponds to EstParameters
# SVMEstConfMat,RVMEstConfMat,GPEstConfMat corresponds to EstConfMatrices
# SVMConfMatrix,RVMConfMatrix,GPConfMatrix corresponds to ConfMatrix
# Calculate and display the time taken for cross validation
start = datetime.datetime.now()
SVM_Y,RVM_Y,GPR_Y,SVM_best_params,RVM_best_params,GPR_best_params,\
SVMEstConfMat,RVMEstConfMat,GPEstConfMat,SVMConfMatrix,RVMConfMatrix,GPConfMatrix\
= MyCrossValidate(x_train_def, y_train_def, 5)

Dis_CV_Info(SVM_best_params,RVM_best_params,GPR_best_params,
SVMEstConfMat,RVMEstConfMat,GPEstConfMat,SVMConfMatrix,RVMConfMatrix,GPConfMatrix)
end = datetime.datetime.now()
print('Time for Cross Validation: ', end=' ')
print (end-start)

The Shape of Support Vectors Array of SVM model for Fold 0 is:
(2070, 60)
The Length of Relevance Vectors Array of RVM model for Fold 0 is:
6
The Shape of Support Vectors Array of SVM model for Fold 1 is:
(2083, 60)
The Length of Relevance Vectors Array of RVM model for Fold 1 is:
6
The Shape of Support Vectors Array of SVM model for Fold 2 is:
(2042, 60)
The Length of Relevance Vectors Array of RVM model for Fold 2 is:
6
The Shape of Support Vectors Array of SVM model for Fold 3 is:
(2042, 60)
The Length of Relevance Vectors Array of RVM model for Fold 3 is:
6
The Shape of Support Vectors Array of SVM model for Fold 4 is:
(2061, 60)
The Length of Relevance Vectors Array of RVM model for Fold 4 is:
6
The parameters of the best SVM model are: 
{'gamma': 1, 'tol': 0.001}
The parameters of the best RVM model are: 
{'kernel': 'linear', 'tol': 0.005}
The parameters of the best GPR model are: 
{'max_iter_predict': 50, 'n_restarts_optimizer': 0}
The Confusion Matrix For Fold 0 is: 
SVM:
[[160